In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, sum as sum_, first, round, min, when, lit, to_date, to_timestamp, date_format
from pyspark.sql.types import DoubleType, IntegerType, StringType, DateType, TimestampType

# Configurações do Spark
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.901')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')

# Criar sessão Spark
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# Ler o CSV, já tratando "NULL" como valor nulo
TabelaCompleta = spark.read.option('delimiter', ',') \
              .option('header', 'true') \
              .option('nullValue', 'null') \
              .csv('s3a://bucket-raw-upa-connect-1/tabela_historico_sensor.csv')

# Ler o CSV, já tratando "NULL" como valor nulo
upa_df = spark.read.option('delimiter', ',') \
              .option('header', 'true') \
              .csv('s3a://bucket-trusted-upa-connect-1/upa.csv')

paciente_df = spark.read.option('delimiter', ',') \
              .option('header', 'true') \
              .csv('s3a://bucket-trusted-upa-connect-1/paciente.csv')

sensor_df = spark.read.option('delimiter', ',') \
              .option('header', 'true') \
              .csv('s3a://bucket-trusted-upa-connect-1/sensor.csv')


unidadeDeMedida_df = spark.read.option('delimiter', ',') \
              .option('header', 'true') \
              .csv('s3a://bucket-trusted-upa-connect-1/UnidadeDeMedida.csv')

# Converter 'valor' para DoubleType e 'fk_upa' para IntegerType
# a = a.withColumn('valor', col('valor').cast(DoubleType())) \
#      .withColumn('fk_paciente', col('fk_paciente').cast(IntegerType()))

# TabelaCompleta.show()
# upa_df.show()
# paciente_df.show(170)
# sensor_df.show()
# unidadeDeMedida_df.show()




:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2b586b5e-c8fc-4c78-a634-3cbab0301ce0;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 354ms :: artifacts dl 11ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 by [com.amazonaws#aws-java-sdk-bundle;1.12.262] in [default]
	---------------------------------------------------------------------
	|     

+-----------+------------------+-----------+---------------+---------+
|id_paciente|              nome|        cpf|data_nascimento|biometria|
+-----------+------------------+-----------+---------------+---------+
|          1|         Ana Souza|22813945182|     2015-05-12|     NULL|
|          2|    Bruno Oliveira|93526748006|     2017-08-23|     NULL|
|          3|    Carlos Pereira|30659423739|     2012-11-30|     NULL|
|          4|      Daniela Lima|15362784900|     2016-03-14|     NULL|
|          5|    Eduardo Santos|96283104517|     2014-07-21|     NULL|
|          6|      Luiza Amaral|23498104750|     2018-03-19|     NULL|
|          7|    Miguel Tavares|84571039216|     2020-07-25|     NULL|
|          8|      Nina Cardoso|49386217048|     2014-10-01|     NULL|
|          9|        Otávio Luz|91620348759|     2013-12-11|     NULL|
|         10|     Pietra Moraes|36284917056|     2019-01-30|     NULL|
|         11|     Rebeca Soares|49218573061|     2015-11-12|     NULL|
|     

In [2]:
# 1. Filtrar pelos sensores desejados e remover valores nulos em 'valor'
df_filtrado_inicial = TabelaCompleta.filter(
    (col("fk_sensor") == 3) | (col("fk_sensor") == 4)
).filter(
    col('valor').isNotNull()
)

# 2. Aplicar filtros de outlier específicos para cada sensor
condicao_sensor_3 = (col('fk_sensor') == 3) & (col('valor') >= 0) & (col('valor') <= 100)
condicao_sensor_4 = (col('fk_sensor') == 4) & (col('valor') >= 34) & (col('valor') <= 42)

df_filtrado_final = df_filtrado_inicial.filter(
    condicao_sensor_3 | condicao_sensor_4
).orderBy("fk_paciente") 

df_filtrado_final = df_filtrado_final.withColumn('valor', col('valor').cast(DoubleType())) \
     .withColumn('fk_paciente', col('fk_paciente').cast(IntegerType()))

# Mostrar o DataFrame filtrado
# df_filtrado_final.show()

df_resultado_final = df_filtrado_final.groupBy('fk_paciente').agg(
    # Pega a menor data_hora para cada paciente (pode ser ajustado para 'first' se preferir)
    min(col('data_hora')).alias('data_hora'),
    # Pega o primeiro fk_upa encontrado para cada paciente (assumindo que fk_upa é o mesmo para o paciente)
    min(col('fk_upa')).alias('fk_upa'), # Usando min aqui para pegar um fk_upa consistente
    # Média da Oximetria (fk_sensor = 3)
    round(avg(when(col('fk_sensor') == 3, col('valor'))), 1).alias('Oximetria_Media'),
    # Média da Temperatura (fk_sensor = 4)
    round(avg(when(col('fk_sensor') == 4, col('valor'))), 1).alias('Temperatura_Media')
).orderBy('fk_paciente')

# Mostrar o DataFrame resultante
# df_resultado_final.show(120)


# --- Adicionando as Colunas "Legenda Gravidade" e "Cor Indicativa" ---

df_com_legenda = df_resultado_final.withColumn('Legenda Gravidade',
    when(col('Temperatura_Media') > 39, 'Febre muito alta')
    .when((col('Temperatura_Media') < 35.3) & (col('Oximetria_Media') >= 87) & (col('Oximetria_Media') <= 94.5), 'Temp. baixa + Oxigenação baixa')
    .when((col('Temperatura_Media') > 37.8) & (col('Oximetria_Media') >= 87) & (col('Oximetria_Media') <= 94.5), 'Temperatura alta + Oxigenação baixa')
    .when(col('Temperatura_Media') > 37.8, 'Temperatura alta')
    .when((col('Oximetria_Media') >= 87) & (col('Oximetria_Media') <= 94.5), 'Oxigenação baixa (>87% e <94.5%)')
    .when(col('Oximetria_Media') < 87, 'Oxigenação muito baixa (<87%)')
    .otherwise('Normal')
)

# Mapeamento de prioridade numérica para a Cor Indicativa
# Definimos a ordem das prioridades e as cores correspondentes
df_resultado_final = df_com_legenda.withColumn('Cor Indicativa',
    when(col('Legenda Gravidade') == 'Febre muito alta', '#ff4500') # Laranja Avermelhado (Red-Orange)
    .when(col('Legenda Gravidade') == 'Temp. baixa + Oxigenação baixa', '#ff4500') # Laranja Avermelhado (Red-Orange)
    .when(col('Legenda Gravidade') == 'Temperatura alta + Oxigenação baixa', '#ff4500') # Laranja Avermelhado (Red-Orange)
    .when(col('Legenda Gravidade') == 'Temperatura alta', '#ffa500') # Laranja (Orange)
    .when(col('Legenda Gravidade') == 'Oxigenação baixa (>87% e <94.5%)', '#a52a2a') # Marrom (Brown)
    .when(col('Legenda Gravidade') == 'Oxigenação muito baixa (<87%)', '#ff4500') # Laranja Avermelhado (Red-Orange)
    .otherwise('#0000ff') # Azul (Blue) - para 'Normal' e outros casos não explicitados
)


## Juntando Dados e Reestruturando Colunas
### 1. Renomear `fk_paciente` para o Nome do Paciente

#`join` com o `paciente_df` e depois excluindo colunas desnecessarias.

df_final = df_resultado_final.join(
    paciente_df,
    df_resultado_final.fk_paciente == paciente_df.id_paciente,
    "left"
).withColumnRenamed("nome", "nome_paciente") \
 .drop("fk_paciente") \
 .drop("id_paciente") \
 .drop("cpf") \
 .drop("data_nascimento") \
 .drop("biometria")

df_final = df_final.withColumn('data_chegada', to_date(col('data_hora'), 'yyyy-MM-dd HH:mm:ss').cast(DateType())) \
                   .withColumn('horario_chegada', date_format(col('data_hora'), 'HH:mm:ss').cast(StringType())) \
                   .drop('data_hora') # Remove a coluna data_hora original

#`join` com o `upa_df` e depois excluindo colunas desnecessarias.

df_final = df_final.join(
    upa_df,
    df_final.fk_upa == upa_df.id_upa,
    "left"
).withColumnRenamed("nome", "nome_da_upa") \
 .drop("id_upa") \
 .drop("cnpj") \
 .drop("telefone") \
 .drop("capacidade_atendimento") \
 .drop("fk_endereco")

# --- Definir a ordem final das colunas ---
colunas_finais_ordenadas = [
    'nome_paciente',
    'Oximetria_Media',
    'Temperatura_Media',
    'Legenda Gravidade',
    'Cor Indicativa',
    'data_chegada',
    'horario_chegada',
    'fk_upa',
    'nome_da_upa'
]

df_final = df_final.select(colunas_finais_ordenadas)


# df_final = df_final.fillna({"Temperatura_Media": 35.5})

# Mostrar o DataFrame resultante
df_final.show(120)





+------------------+---------------+-----------------+--------------------+--------------+------------+---------------+------+--------------------+
|     nome_paciente|Oximetria_Media|Temperatura_Media|   Legenda Gravidade|Cor Indicativa|data_chegada|horario_chegada|fk_upa|         nome_da_upa|
+------------------+---------------+-----------------+--------------------+--------------+------------+---------------+------+--------------------+
|         Ana Souza|           89.5|             37.9|Temperatura alta ...|       #ff4500|  2025-05-24|       08:50:00|     6|    UPA CITY JARAGUÁ|
|    Bruno Oliveira|           94.5|             38.4|Temperatura alta ...|       #ff4500|  2025-05-24|       08:55:00|     2|    UPA 26 DE AGOSTO|
|    Carlos Pereira|           90.1|             38.1|Temperatura alta ...|       #ff4500|  2025-05-24|       09:00:00|    19|UPA LAPA - PROF. ...|
|      Daniela Lima|           92.8|             38.2|Temperatura alta ...|       #ff4500|  2025-05-24|       09

In [3]:
df_final.coalesce(1) \
        .write \
        .option('header', 'true') \
        .mode('overwrite') \
        .csv('s3a://bucket-trusted-upa-connect-1/grafico_oximetria_temperatura.csv')

25/05/25 02:37:50 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
25/05/25 02:37:51 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [4]:
# --- Instalação de bibliotecas (adicione esta seção no início do seu script) ---
import subprocess
import sys

# Função para instalar pacotes
def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"'{package}' instalado com sucesso.")
    except Exception as e:
        print(f"Erro ao instalar '{package}': {e}")
        # Em ambientes de produção, você pode querer adicionar um exit(1) aqui
        # para parar a execução se uma dependência crítica não puder ser instalada.

# Instalar as bibliotecas necessárias
install_package("pandas")
install_package("gspread")
install_package("google-auth-oauthlib") # Necessário para google.oauth2.service_account.Credentials
install_package("boto3") # Para interagir com o S3
# --- Fim da seção de instalação de bibliotecas ---


# --- CÓDIGO PARA ENVIAR PARA O GOOGLE SHEETS ---
import pandas as pd # <-- MOVIDO PARA AQUI PARA GARANTIR QUE 'pd' ESTEJA NO ESCOPO
import gspread
from google.oauth2.service_account import Credentials
import boto3
import os
import sys

# === CONFIGURAÇÕES PARA O GOOGLE SHEETS ===
GOOGLE_SHEET_ID = '1i6BfuZXPOcTp6BFAiVkpktOBym0HtakZacUKfDzu1zI'
S3_BUCKET_CREDENTIALS = 'bucket-trusted-upa-connect-1'
S3_KEY_CREDENTIALS = 'credenciais.json'
LOCAL_CREDENTIALS_PATH = '/tmp/credenciais.json'

# === AUTENTICAÇÃO GOOGLE ===
print(f"Baixando credenciais do S3: s3://{S3_BUCKET_CREDENTIALS}/{S3_KEY_CREDENTIALS} para {LOCAL_CREDENTIALS_PATH}...")
s3 = boto3.client('s3')
try:
    s3.download_file(S3_BUCKET_CREDENTIALS, S3_KEY_CREDENTIALS, LOCAL_CREDENTIALS_PATH)
    print("Credenciais baixadas com sucesso.")
except Exception as e:
    print(f"Erro ao baixar credenciais do S3: {e}")
    sys.exit(1)

scopes = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
credenciais = Credentials.from_service_account_file(LOCAL_CREDENTIALS_PATH, scopes=scopes)
cliente = gspread.authorize(credenciais)
planilha = cliente.open_by_key(GOOGLE_SHEET_ID)

# === ENVIA O DATAFRAME FINAL PARA A NOVA ABA "teste" ===
NOME_ABA_TESTE = "teste"
print(f"Enviando DataFrame final para a aba '{NOME_ABA_TESTE}'...")

# Converter Spark DataFrame para Pandas DataFrame
try:
    df_pandas = df_final.toPandas()
    print("DataFrame Spark convertido para Pandas com sucesso.")
except Exception as e:
    print(f"Erro ao converter Spark DataFrame para Pandas: {e}")
    print("O DataFrame pode ser muito grande para a memória do driver ou a sessão Spark está inválida.")
    sys.exit(1)

# --- NOVO TRECHO: Ajustar tipagem das colunas para o Google Sheets ---
# Apenas formata colunas de data/hora para string ISO 8601.
# As demais colunas (numéricas, strings, etc.) são mantidas em seus tipos nativos no Pandas.
# O gspread e o Google Sheets farão a inferência de tipo para elas.

# Coluna 'data_chegada': assegurar que é string DD/MM/YYYY
if 'data_chegada' in df_pandas.columns:
    if pd.api.types.is_datetime64_any_dtype(df_pandas['data_chegada']):
        df_pandas['data_chegada'] = df_pandas['data_chegada'].dt.strftime('%d/%m/%Y') # <-- MUDANÇA AQUI
    elif pd.api.types.is_string_dtype(df_pandas['data_chegada']):
        # Tentar converter strings que podem não estar em formato padrão para datetime,
        # e depois formatar para DD/MM/YYYY. 'errors=coerce' transforma não-datas em NaT.
        df_pandas['data_chegada'] = pd.to_datetime(df_pandas['data_chegada'], errors='coerce').dt.strftime('%d/%m/%Y') # <-- MUDANÇA AQUI
    # Preencher valores nulos (NaT) com string vazia, pois None também pode causar problemas.
    df_pandas['data_chegada'] = df_pandas['data_chegada'].fillna('')

# Coluna 'horario_chegada': assegurar que é string HH:MM:SS
if 'horario_chegada' in df_pandas.columns:
    if pd.api.types.is_datetime64_any_dtype(df_pandas['horario_chegada']):
        df_pandas['horario_chegada'] = df_pandas['horario_chegada'].dt.strftime('%H:%M:%S')
    elif pd.api.types.is_string_dtype(df_pandas['horario_chegada']):
        # Se já é string (como no seu caso vindo do Spark), apenas garanta que nulos sejam preenchidos.
        df_pandas['horario_chegada'] = df_pandas['horario_chegada'].fillna('')


# Nenhuma conversão global df_pandas.astype(str) é necessária aqui.
# gspread é capaz de lidar com int, float, bool, e strings diretamente.

dados = [df_pandas.columns.tolist()] + df_pandas.values.tolist()

try:
    aba = planilha.worksheet(NOME_ABA_TESTE)
    aba.clear()
    print(f"Aba '{NOME_ABA_TESTE}' encontrada e limpa.")
except gspread.exceptions.WorksheetNotFound:
    aba = planilha.add_worksheet(title=NOME_ABA_TESTE, rows=str(len(dados) + 100), cols=str(len(df_pandas.columns) + 10))
    print(f"Aba '{NOME_ABA_TESTE}' criada.")

try:
    aba.update(dados)
    print(f"Dados do DataFrame final enviados com sucesso para a aba '{NOME_ABA_TESTE}'.")
except Exception as e:
    print(f"Erro ao atualizar a aba do Google Sheets: {e}")

# Opcional: Remover o arquivo de credenciais temporário
if os.path.exists(LOCAL_CREDENTIALS_PATH):
    os.remove(LOCAL_CREDENTIALS_PATH)
    print(f"Arquivo de credenciais temporário '{LOCAL_CREDENTIALS_PATH}' removido.")

# Parar a sessão Spark
# spark.stop()

'pandas' instalado com sucesso.


'gspread' instalado com sucesso.


'google-auth-oauthlib' instalado com sucesso.


'boto3' instalado com sucesso.
Baixando credenciais do S3: s3://bucket-trusted-upa-connect-1/credenciais.json para /tmp/credenciais.json...
Credenciais baixadas com sucesso.
Enviando DataFrame final para a aba 'teste'...


/usr/local/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


DataFrame Spark convertido para Pandas com sucesso.
Aba 'teste' encontrada e limpa.
Dados do DataFrame final enviados com sucesso para a aba 'teste'.
Arquivo de credenciais temporário '/tmp/credenciais.json' removido.


In [5]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import (
#     col, avg, round, first, to_timestamp, unix_timestamp, lit,
#     when, row_number, date_format, sum as sum_, expr, least
# )
# from pyspark.sql.types import DoubleType, IntegerType, TimestampType, LongType
# from pyspark.sql.window import Window
# from datetime import datetime, timedelta

# # Inicializar Spark
# spark = SparkSession.builder.getOrCreate()

# # Constantes dos arquivos (ajustar conforme necessário para seu ambiente S3)
# TEMPERATURA_FILE_KEY = 'temperatura_paciente_tratado.csv/part-00000-6c299ffa-f8d8-401d-8945-211c631ae8ef-c000.csv'
# OXIMETRO_FILE_KEY = 'oximetria_paciente_tratado.csv/part-00000-24fbb7c6-5570-46aa-828a-66d08839c095-c000.csv'
# PACIENTE_FILE_KEY = 'paciente.csv'
# UPA_FILE_KEY = 'upa.csv'

# # Data de referência (para considerar apenas dados até esta data)
# DATA_ATUAL_STR = "2025-04-27"
# # Usamos o final do dia para incluir todos os dados até a data especificada
# DATA_ATUAL_TIMESTAMP = datetime.strptime(DATA_ATUAL_STR + " 23:59:59", "%Y-%m-%d %H:%M:%S")

# print(f"Data de referência para filtro (incluindo dados até): {DATA_ATUAL_TIMESTAMP}")

# # ------------------ Leitura e tratamento ------------------

# print("Lendo e tratando dados de Temperatura...")
# temperatura_df = spark.read.option('header', 'true').csv(f's3a://bucket-trusted-upa-connect-1/{TEMPERATURA_FILE_KEY}')
# temperatura_df = temperatura_df.withColumn('media_temperatura', col('media_temperatura').cast(DoubleType())) \
#                                .withColumn('fk_paciente', col('fk_paciente').cast(IntegerType())) \
#                                .filter(col('media_temperatura').isNotNull()) \
#                                .filter((col('media_temperatura') >= 30) & (col('media_temperatura') <= 45))
# # Agrupa para obter a primeira leitura (potencialmente a chegada) e a média
# temperatura_df = temperatura_df.groupBy('fk_paciente').agg(
#     first('data_hora').alias('data_hora_temp'), # Primeira data/hora de registro da temperatura
#     round(avg('media_temperatura'), 2).alias('media_temperatura')
# )
# print(f"Temperatura DF após tratamento e agregação: {temperatura_df.count()} rows")

# print("Lendo e tratando dados de Oximetria...")
# oximetria_df = spark.read.option('header', 'true').csv(f's3a://bucket-trusted-upa-connect-1/{OXIMETRO_FILE_KEY}')
# oximetria_df = oximetria_df.withColumn('media_oximetria', col('media_oximetria').cast(DoubleType())) \
#                            .withColumn('fk_paciente', col('fk_paciente').cast(IntegerType())) \
#                            .filter(col('media_oximetria').isNotNull()) \
#                            .filter((col('media_oximetria') >= 70) & (col('media_oximetria') <= 100))
# # Agrupa para obter a primeira leitura (potencialmente a chegada) e a média
# oximetria_df = oximetria_df.groupBy('fk_paciente').agg(
#     first('data_hora').alias('data_hora_oxi'), # Primeira data/hora de registro da oximetria
#     round(avg('media_oximetria'), 2).alias('media_oximetria')
# )
# print(f"Oximetria DF após tratamento e agregação: {oximetria_df.count()} rows")

# print("Lendo dados de Paciente...")
# paciente_df = spark.read.option('header', 'true').csv(f's3a://bucket-trusted-upa-connect-1/{PACIENTE_FILE_KEY}')
# paciente_df = paciente_df.withColumn('id_paciente', col('id_paciente').cast(IntegerType())) \
#                          .withColumn('fk_upa', col('fk_upa').cast(IntegerType())) \
#                          .withColumnRenamed('nome', 'nome_paciente')
# print(f"Paciente DF: {paciente_df.count()} rows")

# print("Lendo dados de UPA...")
# upa_df = spark.read.option('header', 'true').csv(f's3a://bucket-trusted-upa-connect-1/{UPA_FILE_KEY}')
# upa_df = upa_df.withColumn('id_upa', col('id_upa').cast(IntegerType())) \
#                .withColumnRenamed('nome', 'nome_upa')
# print(f"UPA DF: {upa_df.count()} rows")

# # ------------------ Junção e enriquecimento ------------------

# print("Realizando junções...")
# # Junta temperatura e oximetria (apenas pacientes com as duas leituras válidas)
# dados_df = temperatura_df.join(oximetria_df, 'fk_paciente', 'inner')
# print(f"DF após junção Temperatura/Oximetria: {dados_df.count()} rows")

# # Junta com dados do paciente para nome e FK UPA
# dados_df = dados_df.join(paciente_df.select('id_paciente', 'nome_paciente', 'fk_upa'),
#                          dados_df.fk_paciente == paciente_df.id_paciente, 'inner') \
#                    .drop('id_paciente')
# print(f"DF após junção com Paciente: {dados_df.count()} rows")

# # Converte datas/horas de string para Timestamp
# dados_df = dados_df.withColumn('data_hora_temp_ts', to_timestamp('data_hora_temp')) \
#                    .withColumn('data_hora_oxi_ts', to_timestamp('data_hora_oxi'))

# # **MELHORIA 1: Determinar a hora de chegada mais precisa**
# # Usa o menor timestamp entre temperatura e oximetria como hora de chegada estimada
# dados_df = dados_df.withColumn('data_hora_chegada_ts', least(col('data_hora_temp_ts'), col('data_hora_oxi_ts')))
# print("Coluna 'data_hora_chegada_ts' criada usando o menor timestamp de temp/oxi.")


# # Filtra pacientes cujas *chegadas estimadas* ocorreram até a data de referência
# dados_df = dados_df.filter(col('data_hora_chegada_ts') <= lit(DATA_ATUAL_TIMESTAMP))
# print(f"DF após filtro por data de referência (baseado na hora de chegada estimada): {dados_df.count()} rows")


# # Junta com dados da UPA para o nome da UPA
# dados_df = dados_df.join(upa_df.select('id_upa', 'nome_upa'),
#                          dados_df.fk_upa == upa_df.id_upa, 'left') \
#                    .drop('id_upa')
# print(f"DF após junção com UPA: {dados_df.count()} rows")


# # ------------------ Classificação e Priorização ------------------

# print("Classificando pacientes e atribuindo prioridade...")
# # Re-aplicando a lógica de classificação e prioridade como estava no código original para consistência
# dados_df = dados_df.withColumn('Cor Indicativa',
#     when(col('media_temperatura') > 39, 'red')
#     .when((col('media_temperatura') < 35.3) & (col('media_oximetria') < 94.5), 'purple')
#     .when((col('media_temperatura') > 37.8) & (col('media_oximetria') < 94.5), 'red')
#     .when(col('media_temperatura') > 37.8, 'orange')
#     .when(col('media_oximetria') < 94.5, 'brown')
#     .otherwise('blue')
# )

# dados_df = dados_df.withColumn('Legenda Gravidade',
#     when(col('media_temperatura') > 39, 'Febre muito alta')
#     .when((col('media_temperatura') < 35.3) & (col('media_oximetria') < 94.5), 'Temp. baixa + Oxigenação baixa')
#     .when((col('media_temperatura') > 37.8) & (col('media_oximetria') < 94.5), 'Temperatura alta + Oxigenação baixa')
#     .when(col('media_temperatura') > 37.8, 'Temperatura alta')
#     .when(col('media_oximetria') < 94.5, 'Oxigenação baixa (<94.5%)')
#     .otherwise('Normal')
# )

# # Mapeamento de prioridade numérica com base na Legenda Gravidade original
# dados_df = dados_df.withColumn('prioridade',
#     when(col('Legenda Gravidade') == 'Febre muito alta', 1)
#     .when(col('Legenda Gravidade') == 'Temp. baixa + Oxigenação baixa', 1)
#     .when(col('Legenda Gravidade') == 'Temperatura alta + Oxigenação baixa', 2)
#     .when(col('Legenda Gravidade') == 'Temperatura alta', 3)
#     .when(col('Legenda Gravidade') == 'Oxigenação baixa (<94.5%)', 4)
#     .otherwise(5)
# )

# # Adicionar tempo base de espera (em minutos) associado à prioridade
# dados_df = dados_df.withColumn('tempo_base',
#     when(col('prioridade') == 1, 30)
#     .when(col('prioridade') == 2, 20)
#     .when(col('prioridade') == 3, 15)
#     .when(col('prioridade') == 4, 10)
#     .otherwise(5)
# )
# print("Classificação, prioridade e tempo base atribuídos.")

# # ------------------ Cálculo da Fila e Tempo de Espera Melhorado ------------------

# print("Calculando posição na fila e tempo de espera por UPA...")

# # Usar a hora de chegada estimada para ordenação
# fila_window = Window.partitionBy('fk_upa').orderBy(col('prioridade'), col('data_hora_chegada_ts'))

# # Calcular a posição na fila
# dados_df = dados_df.withColumn('Posição na fila', row_number().over(fila_window))
# print("'Posição na fila' calculada com base na hora de chegada estimada.")

# # Calcular a soma acumulada do 'tempo_base' dos pacientes *anteriores*
# espera_window = Window.partitionBy('fk_upa').orderBy(col('prioridade'), col('data_hora_chegada_ts')).rowsBetween(Window.unboundedPreceding, -1)

# # Calcular o TEMPO DE ESPERA estimado antes do início do atendimento (em minutos)
# dados_df = dados_df.withColumn('Tempo de Espera (mins)',
#     (when(sum_('tempo_base').over(espera_window).isNull(), 0).otherwise(sum_('tempo_base').over(espera_window)))
# )
# print("'Tempo de Espera (mins)' calculado.")

# # Calcular o Horário Estimado de Atendimento
# # CORRIGIDO: Usando expr com sintaxe SQL de multiplicação por INTERVAL 1 MINUTE
# # Adiciona o Tempo de Espera (em minutos) ao Horário de Chegada Estimada usando sintaxe SQL de multiplicação por intervalo
# dados_df = dados_df.withColumn('data_hora_atendimento_estimada_ts',
#                                 expr("`data_hora_chegada_ts` + `Tempo de Espera (mins)` * INTERVAL 1 MINUTE")) # Multiplica a coluna por um literal INTERVAL
# print("Cálculo da coluna 'data_hora_atendimento_estimada_ts' corrigido usando expr com multiplicação por INTERVAL 1 MINUTE.")


# # Formata o horário estimado de atendimento para exibição
# dados_df = dados_df.withColumn('Hora Atendimento Estimada', date_format(col('data_hora_atendimento_estimada_ts'), 'HH:mm:ss'))
# print("'Horário Estimado de Atendimento' formatado para exibição.")

# # Adicionar a hora de chegada formatada (usando a coluna mais precisa)
# dados_df = dados_df.withColumn('Hora Chegada Estimada', date_format(col('data_hora_chegada_ts'), 'HH:mm:ss'))

# print("Cálculos de fila e espera concluídos.")

# # ------------------ Resultado final ------------------

# print("Preparando resultado final...")
# # Selecionar e renomear colunas para o resultado final
# resultado_df = dados_df.select(
#     col('nome_paciente').alias('Nome Paciente'),
#     round(col('media_temperatura'), 2).alias('Temperatura Média'),
#     round(col('media_oximetria'), 2).alias('Oximetria Média'),
#     col('Cor Indicativa'),
#     col('Legenda Gravidade'),
#     col('prioridade').alias('Prioridade'),
#     col('fk_upa').alias('ID UPA'),
#     col('nome_upa').alias('Nome UPA'),
#     col('Hora Chegada Estimada'), # Usando a nova hora de chegada mais precisa
#     col('Posição na fila'),
#     col('Tempo de Espera (mins)'), # Tempo de espera em minutos
#     col('Hora Atendimento Estimada') # Horário estimado de início do atendimento
# )

# # Ordenar o resultado para visualização (por UPA e Posição na Fila)
# resultado_df = resultado_df.orderBy('ID UPA', 'Posição na fila')

# # Mostrar resultado (limite 20 linhas por padrão no show)
# print("Mostrando resultado (primeiras 20 linhas):")
# resultado_df.show(truncate=False)

# # Mostrar o schema final para ver os tipos de dados
# print("Schema do resultado final:")
# resultado_df.printSchema()

# # Mostrar contagem total do resultado
# print(f"Total de pacientes no resultado: {resultado_df.count()}")


# # Salvar em CSV
# print("Salvando resultado em S3...")
# # coalesce(1) para escrever em um único arquivo CSV (bom para resultados pequenos)
# # mode('overwrite') para substituir o arquivo se ele já existir
# try:
#     resultado_df.coalesce(1) \
#         .write \
#         .option('header', 'true') \
#         .mode('overwrite') \
#         .csv('s3a://bucket-trusted-upa-connect-1/fila_espera_upa_melhorada.csv') # Nome de arquivo para a versão melhorada
#     print("Resultado salvo com sucesso em s3a://bucket-trusted-upa-connect-1/fila_espera_upa_melhorada.csv")
# except Exception as e:
#     print(f"Erro ao salvar no S3: {e}")


# # Finalizar Spark
# print("Parando sessão Spark...")
# spark.stop()
# print("Sessão Spark parada.")